In [35]:
%%script false --no-raise-error

!pip install torchattacks
!pip install pandas
!pip install openpyxl
!pip install numpy
!pip install plot 
!pip install numpy

In [36]:
#Librairies for maths
import numpy as np
import matplotlib.pyplot as plt

#Libararies for deep learning
import torch.nn.functional as F
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torchvision
import numpy as np
from torch.utils.data import DataLoader
import torch.optim as optim

import torchattacks

import math
import time

import random
import pandas as pd
import openpyxl

In [37]:
import DefaultModel as Util
import MnistModel as MnistModel

import Maillard
import CW_L0
import TestUtils

print(dir(Util))
print(dir(MnistModel))


['AddSecurity', 'CalculateNorms', 'HardwareIdentification', 'TestLoop', 'Trainloop', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'datasets', 'get_pred', 'imshow', 'np', 'plt', 'torch', 'torchvision', 'transforms']
['ConvNeuralNet', 'ImportData', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'datasets', 'nn', 'torch', 'transforms']


In [38]:
Device = Util.HardwareIdentification()
TrainDataLoaderMNIST,TestDataLoaderMNIST = MnistModel.ImportData(1000)

cuda
60000
10000


In [39]:
Model = MnistModel.ConvNeuralNet(10)
Model = Model.to(Device)
Epochs = 40
Loss = nn.CrossEntropyLoss()
LearningRate = 0.001
Optimizer = torch.optim.Adam(Model.parameters(), lr=LearningRate)

In [40]:
#%%script false --no-raise-error
for Epoch in range(Epochs):
    ReturnLoss = Util.Trainloop(TrainDataLoaderMNIST, Model, Loss, Optimizer, Device)
    print('{:.4f}'.format(ReturnLoss))
ModelPath = 'ModelMNIST.torch'
torch.save(Model.state_dict(), ModelPath)

In [42]:
Model.load_state_dict(torch.load('ModelMNIST.torch'))

RuntimeError: Error(s) in loading state_dict for ConvNeuralNet:
	size mismatch for F3.weight: copying a param with shape torch.Size([10, 512]) from checkpoint, the shape in current model is torch.Size([1000, 512]).
	size mismatch for F3.bias: copying a param with shape torch.Size([10]) from checkpoint, the shape in current model is torch.Size([1000]).

In [ ]:
Util.TestLoop(TestDataLoaderMNIST, Model, Device)

Accuracy: 99.37%


In [ ]:
%%script false --no-raise-error

ActivationList = [4]
CoefList = [5]
CList = [10.0]
ProbList = [4]
SimilarityList = [0.2]
Init = 0

data = []
for X, y in TestDataLoaderMNIST:
  for S in SimilarityList:
    for P in ProbList:
      for C in CList:
        for Coef in CoefList:
          for Activation in ActivationList:
            DataResult = TestUtils.AttackNorms(Model, C, 1, 0.2, Coef, Activation, P, S, Init, X, y, Device)
            data += DataResult
  break

df = pd.DataFrame(data, columns=['L0 Norm', 'L2 Norm', 'Linf Norm', 'Activation', 'Coef', 'Prob', 'c', 'Similarity', 'Pre', 'Ppred', 'Apred', 'Spre'])
df.to_excel("ResultsM.xlsx")

In [ ]:
%%script false --no-raise-error

Activation = 4
Coef = 5
Prob = 2
Similarity = 0.2
Init=1

X, y = next(iter(TestDataLoaderMNIST))

data = []
start_time = time.time()
data = TestUtils.TestLoopC(Model, 1, 0.2, Coef, Activation, Prob, Similarity, Init, X, y, Device, 10, 1, 2)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time)
TestUtils.SaveData(data, "Best.xlsx")

In [ ]:
Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=2
lr = 0.02

X, y = next(iter(TestDataLoaderMNIST))

TestedIterations = [100000]

for i in range((len(TestedIterations))):
    data = []
    start_time = time.time()
    data = TestUtils.TestLoopC(Model, TestedIterations[i], lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5)
    end_time = time.time()

    TestUtils.SaveTime(start_time, end_time, "TimeXM" + str(TestedIterations[i]) +".txt")
    TestUtils.SaveData(data, "BestXM" + str(TestedIterations[i]) + ".xlsx")
    

492 15.5041485 0.97678393 4 5 1000 2 0.2 7 7 2 7
532 15.582347 0.9686122 4 5 1000 2 0.2 2 2 7 2
501 15.424045 0.97459376 4 5 1000 2 0.2 1 1 2 1
502 15.279991 0.9737092 4 5 1000 2 0.2 0 0 9 0
516 15.580674 0.9730423 4 5 1000 2 0.2 4 4 2 4
495 15.740185 0.9838682 4 5 1000 2 0.2 1 1 2 1
475 15.293172 0.9810689 4 5 1000 2 0.2 4 4 2 4
490 15.453863 0.9765632 4 5 1000 2 0.2 9 9 2 9
570 16.025066 0.98233145 4 5 1000 2 0.2 5 5 0 5
488 15.411925 0.98576623 4 5 1000 2 0.2 9 9 2 9
502 15.51846 0.9823311 4 5 1000 2 0.2 0 0 2 0
394 14.269554 0.9837165 4 5 1000 2 0.2 6 6 2 6
492 15.165697 0.96289396 4 5 1000 2 0.2 9 9 2 9
465 14.842844 0.9697952 4 5 1000 2 0.2 0 0 2 0
609 17.08067 0.9677936 4 5 1000 2 0.2 1 1 2 1
468 15.198235 0.97323257 4 5 1000 2 0.2 5 5 2 5
447 15.30076 0.97757363 4 5 1000 2 0.2 9 9 2 9
475 15.140142 0.95217884 4 5 1000 2 0.2 7 7 2 7
436 14.3470545 0.96723175 4 5 1000 2 0.2 3 3 2 3
535 15.598577 0.9769486 4 5 1000 2 0.2 4 4 2 4
412 14.14938 0.9689358 4 5 1000 2 0.2 9 9 2 9
504 14

KeyboardInterrupt: 

In [ ]:
%%script false --no-raise-error

Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=1
lr = 0.02

X, y = next(iter(TestDataLoaderMNIST))
TestUtils.SaveLoopC(Model, 100000, lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5, "MNIST_images/")

20 4.08623 0.99999964 4 5 1000 2 0.2 7 7 1 7
14 3.6881657 0.99959004 4 5 1000 2 0.2 2 2 8 2
8 2.6482923 0.9998514 4 5 1000 2 0.2 1 1 5 1
13 3.4860575 0.99933964 4 5 1000 2 0.2 0 0 8 0
8 2.0814967 0.9249069 4 5 1000 2 0.2 4 4 9 4
22 4.1384315 1.0 4 5 1000 2 0.2 1 1 6 1
17 3.8564806 0.99999833 4 5 1000 2 0.2 4 4 5 4
10 2.977239 1.0 4 5 1000 2 0.2 9 9 1 9
11 2.9504015 1.0 4 5 1000 2 0.2 5 5 6 5
13 3.4971128 0.9999979 4 5 1000 2 0.2 9 9 4 9
21 3.9126215 0.9999788 4 5 666.6666666666666 2 0.2 7 7 1 7
13 3.5290854 0.9986644 4 5 666.6666666666666 2 0.2 2 2 1 2
13 3.1970067 0.99984646 4 5 666.6666666666666 2 0.2 1 1 4 1
11 3.1635103 0.99897015 4 5 666.6666666666666 2 0.2 0 0 9 0
11 3.2879288 0.9995674 4 5 666.6666666666666 2 0.2 4 4 9 4
15 3.4780414 0.9999951 4 5 666.6666666666666 2 0.2 1 1 2 1
17 3.9792366 0.9999985 4 5 666.6666666666666 2 0.2 4 4 1 4
12 3.008775 0.9986067 4 5 666.6666666666666 2 0.2 9 9 7 9
9 2.7420242 0.9995967 4 5 666.6666666666666 2 0.2 5 5 6 5
14 3.577104 0.9998472 4 5 66

In [ ]:
%%script false --no-raise-error

dataJSMA = []

X, y = next(iter(TestDataLoaderMNIST))

start_time = time.time()
Data = TestUtils.TestJSMA(1,1, Model, X, y, Device)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "TimeJSMAM" + str(TestedIterations[i]) +".txt")
TestUtils.SaveData(Data, "BestJSMAM.xlsx")

720 25.784443 1.0 0 0 0 0 0 7 0 0 0
[[720, array(25.784443, dtype=float32), array(1., dtype=float32), 0, 0, 0, 0, 0, 7, 0, 0, 0]]


: 

In [ ]:
%%script false --no-raise-error

Count=0
X, y = next(iter(TestDataLoaderMNIST))
dataCW = []

start_time = time.time()
Data = TestUtils.TestCW(1000, 1, 15, Model, X, y, Device)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "TimeCWM" + str(TestedIterations[i]) +".txt")
TestUtils.SaveData([Data], "BestCWM.xlsx")

17 4.480908 1.4505616 0 0 0 0 0 7 7 5 7
4 2.6064174 1.3251966 0 0 0 0 0 2 2 0 2
27 4.963036 1.4876779 0 0 0 0 0 1 1 7 1
5 2.3945875 1.0972978 0 0 0 0 0 0 0 9 0
4 2.4594944 1.4229655 0 0 0 0 0 4 4 9 4
[[17, array(4.480908, dtype=float32), array(1.4505616, dtype=float32), 0, 0, 0, 0, 0, 7, 7, 5, 7], [4, array(2.6064174, dtype=float32), array(1.3251966, dtype=float32), 0, 0, 0, 0, 0, 2, 2, 0, 2], [27, array(4.963036, dtype=float32), array(1.4876779, dtype=float32), 0, 0, 0, 0, 0, 1, 1, 7, 1], [5, array(2.3945875, dtype=float32), array(1.0972978, dtype=float32), 0, 0, 0, 0, 0, 0, 0, 9, 0], [4, array(2.4594944, dtype=float32), array(1.4229655, dtype=float32), 0, 0, 0, 0, 0, 4, 4, 9, 4]]


: 